In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import re
url = 'https://raw.githubusercontent.com/patriciof3/mentoria13_famaf_2024/main/df_reddit_mentoria.csv'
df = pd.read_csv(url)

# **Trabajo Práctico 2**

Después de un primer acercamiento, es probable que hayamos identificado algunos problemas en nuestra base de datos. El objetivo del presente práctico es solucionarlos definitivamente y dejar listo un dataset que nos permita trabajar de manera correcta con diferentes modelos de clasificación o clusterización. Para ello deberemos realizar una limpieza adecuada de la base, que descarte entradas nulas, spam, texto en otros idiomas y stopwords.

## Stemming y Lemmatization

Luego lo que debe hacerse es un proceso de normalización de palabras, es decir, reducir la variabilidad que existe entre vocablos de la misma familia.

Para ello existen en principio dos técnicas diferentes:

- Stemming: Cortar las palabras para dejar solo su raíz
- Lemmatización: Se utilizan librerías con funciones preentrenadas

In [6]:
# tomamos una entrada y removemos caracteres especiales
example = re.sub(r'[^a-zA-ZáéíóúüñÁÉÍÓÚÜÑ\s]', '', df["content"][130])
example

'Empecé con esto desde hace ya tiempo  años las razones son variadas ya que hay historial de enfermedades mentales en mi familia y pase por un momento duro yo sola en aquel entonces Pedi ayuda  años después y estuve medicada por un período muy corto de tiempo He sentido de todo desde las sensaciones más leves hasta las que tengo día con día En particular sufro de mareos y cuando estoy en la escuela tengo dificultad para respirar Mi punto es ustedes creen que haya alguien superado esto por completo Había aceptado vivir asi pero despues empecé a comparar mi vida con la de otras personas y quisiera al menos vivir sin tener esta mentalidad Hace no mucho tiempo logré controlarme Estaba a punto de explotar en una plaza comercial y simplemente no me lo permití Tuve una muy buena semana después de eso pero estar en calma es tan agotador  más porque nunca pensé que tuviera que esforzarme en ser feliz'

In [ ]:
import nltk
from nltk.stem.snowball import SnowballStemmer
import spacy

nlp = spacy.load("es_core_news_sm")

# remover stopwords

stop_words = spacy.lang.es.stop_words.STOP_WORDS

def preprocess_text(text):
    doc = nlp(text)
    tokens = [token.text for token in doc if token.text.lower() not in stop_words]
    return ' '.join(tokens)

example_stopwords = preprocess_text(example)

doc = nlp(example_stopwords)

tokenized = tokens = [token.text for token in doc]


# Stemming
stemmer = SnowballStemmer('spanish')

stemmed_words = [stemmer.stem(word) for word in tokenized]

# Lematización

lemmatized_words = [token.lemma_ for token in doc]

df_example = pd.DataFrame(
    {'Original': tokenized,
     'Stemming': stemmed_words,
     'Lemmatization': lemmatized_words
    })


In [8]:
df_filtered = df_example[df_example.nunique(axis=1) == df_example.shape[1]]
df_filtered

,Original,Stemming,Lemmatization
0,Empecé,empec,empezar
4,razones,razon,razón
5,variadas,vari,variado
10,pase,pas,pasar
14,ayuda,ayud,ayudar
17,estuve,estuv,estar
18,medicada,medic,medicado
23,sensaciones,sensacion,sensación
24,leves,lev,leve
32,creen,cre,creer


## Vectorización

Una vez que tenemos un df apropiadamente curado, podemos pasar a realizar la vectorización de nuestros datos. Bag of Words (BoW) y TF-IDF (Term Frequency-Inverse Document Frequency) son dos técnicas fundamentales utilizadas en el procesamiento de lenguaje natural (NLP) para la representación de textos.

### Bag of Words

Representa el texto como una bolsa de sus palabras, sin tener en cuenta el orden de las palabras ni su gramática. Cada documento se convierte en un vector de frecuencias de palabras, donde cada dimensión del vector corresponde a una palabra del vocabulario.

### TF-IDF
Pondera las palabras en un documento basado en su frecuencia en el documento (TF) y en la frecuencia inversa de documentos (IDF) que contienen esa palabra en el corpus. La idea es reducir la importancia de las palabras comunes y resaltar las palabras raras pero relevantes. Un valor alto de TF-IDF indica que el término es muy relevante para ese documento en particular y no es común en el corpus. Un valor bajo indica que el término no es muy relevante para ese documento específico, o es muy común en el corpus y, por lo tanto, no aporta mucha información.

In [9]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
pd.set_option('display.max_columns', None)
pd.options.display.float_format = "{:,.2f}".format

# Crear DataFrame dummy con comentarios de Reddit
data = {
    'content': [
        'Me siento muy ansioso últimamente',
        'No puedo dormir debido a la ansiedad',
        'Me siento deprimido y sin energía',
        'La depresión está arruinando mi vida',
        'Estoy teniendo ataques de pánico frecuentemente',
        'La tristeza es constante y no sé qué hacer'
    ],
    'subreddit': ['Ansiedad', 'Ansiedad', 'Depresión', 'Depresión', 'Ansiedad', 'Depresión']
}

df_example = pd.DataFrame(data)

def preprocess(text):
    doc = nlp(text)
    return ' '.join([token.lemma_ for token in doc if not token.is_stop and not token.is_punct])

# Aplicar la función de preprocesamiento a la columna de contents
df_example['content'] = df_example['content'].apply(preprocess)

# Implementación de Bag of Words
vectorizer_bow = CountVectorizer()
X_bow = vectorizer_bow.fit_transform(df_example['content'])

# Convertir a DataFrame para mejor visualización
df_bow = pd.DataFrame(X_bow.toarray(), columns=vectorizer_bow.get_feature_names_out())

# Implementación de TF-IDF
vectorizer_tfidf = TfidfVectorizer()
X_tfidf = vectorizer_tfidf.fit_transform(df_example['content'])

# Convertir a DataFrame para mejor visualización
df_tfidf = pd.DataFrame(X_tfidf.toarray(), columns=vectorizer_tfidf.get_feature_names_out())

# Mostrar los DataFrames
print("Bag of Words (BoW):")
display(df_bow)

print("\nTF-IDF:")
display(df_tfidf)

Bag of Words (BoW):


,ansiedad,ansioso,arruinar,ataque,constante,depresión,deprimido,dormir,energía,frecuentemente,pánico,sentir,tener,tristeza,vida,últimamente
0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1
1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0
3,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0
4,0,0,0,1,0,0,0,0,0,1,1,0,1,0,0,0
5,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0



TF-IDF:


,ansiedad,ansioso,arruinar,ataque,constante,depresión,deprimido,dormir,energía,frecuentemente,pánico,sentir,tener,tristeza,vida,últimamente
0,0.00,0.61,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.50,0.00,0.00,0.00,0.61
1,0.71,0.00,0.00,0.00,0.00,0.00,0.00,0.71,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,0.00,0.00,0.00,0.00,0.00,0.00,0.61,0.00,0.61,0.00,0.00,0.50,0.00,0.00,0.00,0.00
3,0.00,0.00,0.58,0.00,0.00,0.58,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.58,0.00
4,0.00,0.00,0.00,0.50,0.00,0.00,0.00,0.00,0.00,0.50,0.50,0.00,0.50,0.00,0.00,0.00
5,0.00,0.00,0.00,0.00,0.71,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.71,0.00,0.00


## Modelado de Tópicos

In [11]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

import pyLDAvis.lda_model


vectorizer = CountVectorizer(max_features=1000, lowercase=True)
X = vectorizer.fit_transform(df_example['content'])


num_topics = 5 # select number of topics
lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda.fit(X)


df_example['topic_probabilities'] = lda.transform(X).tolist()

pyLDAvis.enable_notebook()
prueba = pyLDAvis.lda_model.prepare(lda, X, vectorizer, mds='tsne', n_jobs=1)
pyLDAvis.save_html(prueba, "prueba.html")

c:\Users\patricio\anaconda3\lib\site-packages\sklearn\manifold\_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(
c:\Users\patricio\anaconda3\lib\site-packages\sklearn\manifold\_t_sne.py:819: FutureWarning: 'square_distances' has been introduced in 0.24 to help phase out legacy squaring behavior. The 'legacy' setting will be removed in 1.1 (renaming of 0.26), and the default setting will be changed to True. In 1.3, 'square_distances' will be removed altogether, and distances will be squared by default. Set 'square_distances'=True to silence this warning.
  warnings.warn(
